# Aprendizaje de Máquina
## Tarea 1: clasificación y regresión en vinos
__Descripción:__ En este trabajo realizaremos un modelo de aprendizaje automatizado, para predecir la puntuación de calidad de un vino dependiendo de sus cualidades tales como acidez y azúcares, entre otros datos.

## Implementación de librerias
Importamos las librerias con las que vamos a trabajar en este notebook, entre las cuales podemos apreciar numpy para trata numérica, matplotlib para mostrar gráficas, pandas para trata de DataFrames, scikit-learn para los modelos de aprendizaje y ucimlrepo para importar el dataset de los vinos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
from ucimlrepo import fetch_ucirepo

## Importación del dataset
Importamos el dataset de vinos con el que vamos a trabajar.

In [ ]:
wine_quality = fetch_ucirepo(id=186)

## Transformación del dataset a pandas DataFrame
Transformamos el dataset a un DataFrame de pandas para despues poderlo procesar con la libreria scikit-learn.

In [ ]:
wq_df = pd.DataFrame(data=wine_quality.data.features, columns=wine_quality.data.feature_names)
wq_df['quality'] = wine_quality.data.targets

## Generación de datos sintéticos
Generamos datos sintéticos con la finalidad de aumentar la cantidad de información con la que trabajarán los modelos, esperando obtener mejores resultados de esta manera.
Copiamos 3503 datos del dataset original, para así llegar a un total de 10000 datos, a estos datos nuevos les añadimos un poco de ruido para generar variedad, este ruido no se añade a la columna objetivo de 'Calidad' ya que los datos originales son números enteros.
Una vez generados los datos sintéticos los juntamos con los datos originales para obtener un solo DataFrame con el cuál trabajar por el resto del proyecto.

In [ ]:
shuffled_df = shuffle(wq_df)

synthetic_data = shuffled_df.iloc[:3503].copy()

for column in synthetic_data.select_dtypes(include=np.number):
    if column == 'quality':
        continue
    synthetic_data[column] += np.random.normal(0, 0.1, len(synthetic_data))

combined_data = pd.concat([wq_df, synthetic_data], ignore_index=True)

## Separación de datos de entrenamiento y validación
Generamos dos DataFrames nuevos, uno que contiene todos los datos a excepción de los resultados (la columna de 'calidad'), y otro que solamente contiene los resultados, con este par de DataFrames generamos cuatro DataFrames más:
1. __X_train:__ datos para entrenar los modelos
1. __X_test:__ datos con los que los modelos no trabajan, con la finalidad de usarlos en la fase de evaluación
1. __y_train:__ datos de resultados con los cuales los modelos aprenden a predecir en base a la información entregada previamente
1. __y_test:__ datos de resultados con los cuales los modelos no se entrenan, con la finalidad de evaluar los resultados en la fase de evaluación

In [ ]:
X = combined_data.drop('quality', axis=1)
y = combined_data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Una vez que tenemos todo lo necesario para entrenar a los modelos, procederemos con el entrenamiento para regresión.

# Regresión
La regresión para este caso consiste en generar una relación entre las características del vino, con los resultados, con la finalidad de predecir el mejor resultado posible cuando se entreguen nuevos datos.
Si la clasificación original de una nueva entrada es 3, se entiende por mejor resultado de clasificación un 2 o un 4 que un 7.

## Definición de hiperparámetros para optimización
Definimos una serie de hiperparámetros para un modelo de Support Vector Machine, con los cuales, se probaran todas las combinaciones posibles con la finalidad de encontrar la mejor combinación.
Los hiperparámetros son:
1. __kernel:__ el tipo de transformación kernel que se realizará para trabajar con múltiples dimensiones
1. __C:__ el parámetro de regularización, mientras más grande sea el valor, más se penalizan los errores
1. __gamma:__ coeficiente de kernel para poly, rbf y sigmoid
1. __epsilon:__ la holgura que se le entrega al modelo para poder fallar
1. __max_iter:__ la cantidad máxima de iteraciones que realizará el modelo, con esto evitamos que trabaje de manera indefinida

In [ ]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.01, 0.1, 1],
    'max_iter': [50, 100, 500]
}

## Búsqueda de hiperparámetros
Utilizamos la implementación de GridSearchCV entregada por la librería de scikit-learn.
Esto realiza un entrenamiento con el modelo SVR (Support Vector Regression), probando cada combinación de los hiperparámetros definidos anteriormente, usando validación cruzada y buscando el mejor puntaje $r²$.
Una vez terminado se puede obtener la mejor combinación de parámetros y guardar ese modelo sin necesidad de definirlo a mano.

In [ ]:
svr = SVR()
grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_svr = grid_search.best_estimator_
print("Best hyperparameters", best_params)

## Métricas y predicciones
Una vez obtenido la mejor versión de SVR para este caso, podemos ver las métricas tales como el error cuadrático medio, error absoluto medio y puntaje $r²$.
Además podemos mostrar en un gráfico las predicciones realizadas por el modelo contra los verdaderos resultados de los datos de validación.

In [ ]:
print(f"MSE: {mean_squared_error(y_test, y_pred_best)}")
print(f"MSE**0.5: {mean_squared_error(y_test, y_pred_best)**0.5}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_best)}")
print(f"r2_score: {r2_score(y_test, y_pred_best)}")

plt.scatter(y_test, y_pred_best)
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.title('Best SVR Model Prediction')
plt.show()

# Clasificación
Los modelos de clasificación, como su nombre indica, clasifican los datos en clases, para este caso cada clase es la posible nota que reciba el vino, respecto a los resultados solo hay dos posibilidades, bien clasificado o mal clasificado.
Procederemos a realizar el mismo procedimiento que realizamos para regresión, pero con el modelo que utilizaremos para clasificación.
Debido a la similitud de los procesos solo se hablará de las diferencias.

## Definición de hiperparámetros
El modelo SVC no utiliza epsilon y se puede entregar el valor random_state, que permite definir una semilla para controlar la generación de valores pseudo aleatorios, si no definimos este valor, en cada iteración que se realice se cambiarán ciertos valores llevando a una inconsistencia en el trabajo.

In [ ]:
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100], #Regularization
    'gamma': ['scale', 'auto', 0.1, 1], #kernel coefficient for rbf, poly & sigmoid
    'random_state': [42],
    'max_iter': [50, 100, 500]
}

## Búsqueda de hiperparámetros

In [ ]:
svc = SVC()
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_svc = grid_search.best_estimator_
print("Best hyperparameters", best_params)

## Métricas y predicciones

In [ ]:
print(f"MSE: {mean_squared_error(y_test, y_pred_best)}")
print(f"MSE**0.5: {mean_squared_error(y_test, y_pred_best)**0.5}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_best)}")
print(f"r2_score: {r2_score(y_test, y_pred_best)}")

plt.scatter(y_test, y_pred_best)
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.title('Best SVC Model Prediction')
plt.show()